# Outlier Removal Using Quantiles

This notebook demonstrates a common data cleaning technique: removing outliers using their quantile. Outliers can skew statistical analyses and degrade the performance of ml models. I will explore this method with two different datasets.

**Key Concepts:**
- **Outlier:** a data point that is significantly different from other observations in a dataset.
- **Quantile:** A value that divides a set of data into equal-sized groups

## Example 1: Simple Heights Dataset

I will start with a simple dataset containing names and heights. My goal is to identify and remove entries that are unrealistically high or low for human heights.

In [18]:
import pandas as pd
import zipfile

In [2]:
CSV_PATH = 'data/heights.csv'

df = pd.read_csv(CSV_PATH)
df.head()

,name,height
0,mohan,5.9
1,maria,5.2
2,sakib,5.1
3,tao,5.5
4,virat,4.9


### Identifying Outliers with Quantiles

I'll define the outlier thresholds using quantiles. A common statistical practice is to treat the bottom 5% and top 5% of the data as outliers.

In [3]:
# considering anything in the bottom 5% or top 5% of heights as an outlier.
lower_threshold = df['height'].quantile(0.05)
upper_threshold = df['height'].quantile(0.95)

print(f"Lower Threshold (5th percentile): {lower_threshold:.2f}")
print(f"Upper Threshold (95th percentile): {upper_threshold:.2f}")

Lower Threshold (5th percentile): 3.61
Upper Threshold (95th percentile): 9.69


### Viewing the Identified Outliers

Use the calculated thresholds to see which rows are considered outliers.

In [6]:
# Display data points above the upper threshold
print("Upper Outliers:")
display(df[df['height'] > upper_threshold])

Upper Outliers:


,name,height
9,imran,14.5


In [7]:
# Display data points below the lower threshold
print("\nLower Outliers:")
display(df[df['height'] < lower_threshold])


Lower Outliers:


,name,height
12,yoseph,1.2


### Removing the Outliers

With the outliers identified, I can create a new DataFrame that excludes these values.

In [9]:
# new DataFrame without the outliers
df_no_outliers = df[(df['height'] > lower_threshold) & (df['height'] < upper_threshold)]

print("Original DataFrame shape:", df.shape)
print("DataFrame shape with no outlier:", df_no_outliers.shape)

# Display the cleaned data
df_no_outliers

Original DataFrame shape: (14, 2)
DataFrame shape with no outlier: (12, 2)


,name,height
0,mohan,5.9
1,maria,5.2
2,sakib,5.1
3,tao,5.5
4,virat,4.9
5,khusbu,5.4
6,dmitry,6.2
7,selena,6.5
8,john,7.1
10,jose,6.1


## Example 2: Bengaluru House Price Dataset

I will now use a more complex, real-world dataset. This dataset contains information about house prices in Bengaluru. I'm focusing on the `price_per_sqft` column, which can be prone to outliers due to data entry errors or unique, luxury properties.

In [10]:
CSV_PATH = 'data/bhp.csv'
df = pd.read_csv(CSV_PATH)
df.head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250


In [11]:
print(f"\nDataset Shape: {df.shape}")

# Use describe() to get a statistical summary of the numerical columns
print("\nStatistical Summary:")
display(df.describe())


Dataset Shape: (13200, 7)

Statistical Summary:


,total_sqft,bath,price,bhk,price_per_sqft
count,13200.000000,13200.000000,13200.000000,13200.000000,1.320000e+04
mean,1555.302783,2.691136,112.276178,2.800833,7.920337e+03
std,1237.323445,1.338915,149.175995,1.292843,1.067272e+05
min,1.000000,1.000000,8.000000,1.000000,2.670000e+02
25%,1100.000000,2.000000,50.000000,2.000000,4.267000e+03
50%,1275.000000,2.000000,71.850000,3.000000,5.438000e+03
75%,1672.000000,3.000000,120.000000,3.000000,7.317000e+03
max,52272.000000,40.000000,3600.000000,43.000000,1.200000e+07


### Analyzing the Statistical Summary

The output of `df.describe()` gives insight about potential outliers:
- The **mean** is about 7,920.
- The **standard deviation** is very large, suggesting a wide spread of values.
- The **75th percentile** is 7,317, but the **max** value is extremely high at 12,000,000

This huge gap between the 75th percentile and the maximum value is a clear indicator of outliers.

In [12]:
'''
  Given the extreme nature of the outliers, I'll use a more aggressive quantile range. Instead of the 5th and 95th percentiles, I will use the **0.1th** and **99.9th** percentiles. This will allow me to trim the most extreme values from both ends of the distribution.
'''

lower_threshold, upper_threshold = df['price_per_sqft'].quantile([0.001, 0.999])
print(f"Lower Threshold (0.1th percentile): {lower_threshold:.2f}")
print(f"Upper Threshold (99.9th percentile): {upper_threshold:.2f}")

Lower Threshold (0.1th percentile): 1366.18
Upper Threshold (99.9th percentile): 50959.36


In [13]:
print("Upper Outliers:")
df[df['price_per_sqft'] > upper_threshold]

Upper Outliers:


,location,size,total_sqft,bath,price,bhk,price_per_sqft
345,other,3 Bedroom,11.0,3.0,74.0,3,672727
1005,other,1 BHK,15.0,1.0,30.0,1,200000
1106,other,5 Bedroom,24.0,2.0,150.0,5,625000
4044,Sarjapur Road,4 Bedroom,1.0,4.0,120.0,4,12000000
4924,other,7 BHK,5.0,7.0,115.0,7,2300000
5911,Mysore Road,1 Bedroom,45.0,1.0,23.0,1,51111
6356,Bommenahalli,4 Bedroom,2940.0,3.0,2250.0,4,76530
7012,other,1 BHK,650.0,1.0,500.0,1,76923
7575,other,1 BHK,425.0,1.0,750.0,1,176470
7799,other,4 BHK,2000.0,3.0,1063.0,4,53150


In [14]:
print("Lower Outliers:")
df[df['price_per_sqft'] < lower_threshold]

Lower Outliers:


,location,size,total_sqft,bath,price,bhk,price_per_sqft
665,Yelahanka,3 BHK,35000.0,3.0,130.0,3,371
798,other,4 Bedroom,10961.0,4.0,80.0,4,729
1867,other,3 Bedroom,52272.0,2.0,140.0,3,267
2392,other,4 Bedroom,2000.0,3.0,25.0,4,1250
3934,other,1 BHK,1500.0,1.0,19.5,1,1300
5343,other,9 BHK,42000.0,8.0,175.0,9,416
5417,Ulsoor,4 BHK,36000.0,4.0,450.0,4,1250
5597,JP Nagar,2 BHK,1100.0,1.0,15.0,2,1363
7166,Yelahanka,1 Bedroom,26136.0,1.0,150.0,1,573
7862,JP Nagar,3 BHK,20000.0,3.0,175.0,3,875


### Removing the Outliers and Verifying

Now I'll filter the DataFrame to remove these extreme outliers

In [16]:
# Create a new DataFrame without the extreme outliers
df_no_outliers = df[(df['price_per_sqft'] > lower_threshold) & (df['price_per_sqft'] < upper_threshold)]

print("Original DataFrame shape:", df.shape)
print("DataFrame shape after removing outliers:", df_no_outliers.shape)
print(f"Removed {df.shape[0] - df_no_outliers.shape[0]}.")

Original DataFrame shape: (13200, 7)
DataFrame shape after removing outliers: (13172, 7)
Removed 28.


In [17]:
print("Random Sample from Cleaned Data:")
df_no_outliers.sample(5)

Random Sample from Cleaned Data:


,location,size,total_sqft,bath,price,bhk,price_per_sqft
2765,Electronic City,3 BHK,880.0,2.0,18.0,3,2045
32,Hebbal,3 BHK,1760.0,2.0,123.0,3,6988
2209,Bellandur,3 BHK,1605.0,3.0,85.0,3,5295
9447,other,5 Bedroom,10000.0,5.0,1950.0,5,19500
9669,Brookefield,2 BHK,1389.0,2.0,94.0,2,6767


## Example 3: New York City Airbnb Listings

For the final example, I'll be work with a dataset of Airbnb listings from New York City. My goal is to clean the `price` column. In this scenario, I want to remove both the extremely high-priced listings and listings that are priced very low, which might be errors or not relevant for the specific analysis.

In [ ]:
''' 
ZIP_PATH = "data/airbnb.zip"
DESTINATION_DIR = "data/"

try:
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(DESTINATION_DIR)
    print(f"Success.")
except FileNotFoundError:
    print(f"'{ZIP_PATH}' not found.")
except zipfile.BadZipFile:
    print(f"'{ZIP_PATH}' is not a valid ZIP file.")
except Exception as e:
    print(f"Unexpected error: {e}")
'''

Success.


In [19]:
CSV_PATH = "data/new_york_listings_2024.csv"
df = pd.read_csv(CSV_PATH)
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,rating,bedrooms,beds,baths
0,1312228,Rental unit in Brooklyn · ★5.0 · 1 bedroom,7130382,Walter,Brooklyn,Clinton Hill,40.683710,-73.964610,Private room,55.0,...,2015-12-20,0.03,1,0,0,No License,5.0,1,1,Not specified
1,45277537,Rental unit in New York · ★4.67 · 2 bedrooms ·...,51501835,Jeniffer,Manhattan,Hell's Kitchen,40.766610,-73.988100,Entire home/apt,144.0,...,2023-05-01,0.24,139,364,2,No License,4.67,2,1,1
2,971353993633883038,Rental unit in New York · ★4.17 · 1 bedroom · ...,528871354,Joshua,Manhattan,Chelsea,40.750764,-73.994605,Entire home/apt,187.0,...,2023-12-18,1.67,1,343,6,Exempt,4.17,1,2,1
3,3857863,Rental unit in New York · ★4.64 · 1 bedroom · ...,19902271,John And Catherine,Manhattan,Washington Heights,40.835600,-73.942500,Private room,120.0,...,2023-09-17,1.38,2,363,12,No License,4.64,1,1,1
4,40896611,Condo in New York · ★4.91 · Studio · 1 bed · 1...,61391963,Stay With Vibe,Manhattan,Murray Hill,40.751120,-73.978600,Entire home/apt,85.0,...,2023-12-03,0.24,133,335,3,No License,4.91,Studio,1,1


In [22]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license', 'rating',
       'bedrooms', 'beds', 'baths'],
      dtype='object')

In [21]:
# Perform initial exploration
print(f"Dataset Shape: {df.shape}")
print("\nStatistical Summary for Price:")
display(df['price'].describe())

Dataset Shape: (20758, 22)

Statistical Summary for Price:


count     20758.000000
mean        187.776616
std        1022.797208
min          10.000000
25%          80.000000
50%         125.000000
75%         199.000000
max      100000.000000
Name: price, dtype: float64

### Analyzing Price Distribution

From the summary of the `price` column, I can see:
- The **minimum** price is \$10.
- The **75th percentile** is \$199.
- The **maximum** price is an extreme \$100,000.

This confirms the presence of significant upper-end outliers.

The filtering strategy will be a two-part condition:
1.  **Domain-Based Rule:** I will remove all listings with a `price` of $100 or less. Hypothetically, this could be a business requirement to focus on a specific market segment.
2.  **Statistical Rule:** I  will remove listings in the top 1% of prices to handle the extreme upper outliers. I'll calculate the 99th percentile (`0.99` quantile) to set this upper bound.

In [26]:
upper_threshold = df['price'].quantile(0.99)
print(f"Upper Threshold (99th percentile): ${upper_threshold:.2f}")

# Apply the combined filter to create the final DataFrame
df_filtered = df[
    (df['price'] > 100) & 
    (df['price'] < upper_threshold)
]
df_filtered.sample(10)

Upper Threshold (99th percentile): $999.00


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,rating,bedrooms,beds,baths
8112,10701027,Rental unit in Brooklyn · ★4.96 · 1 bedroom · ...,345252,Thomas,Brooklyn,Clinton Hill,40.680750,-73.962280,Entire home/apt,175.0,...,2023-09-01,0.28,1,62,1,No License,4.96,1,1,1
11032,7594252,Rental unit in Brooklyn · ★5.0 · 2 bedrooms · ...,13283118,Nathalie,Brooklyn,South Slope,40.664190,-73.987380,Entire home/apt,200.0,...,2023-08-14,0.05,1,217,1,No License,5.0,2,2,2
13539,15462208,Condo in New York · ★4.72 · 2 bedrooms · 3 bed...,26073602,Anna,Manhattan,East Harlem,40.794840,-73.949570,Entire home/apt,165.0,...,2019-01-12,1.15,1,0,0,No License,4.72,2,3,1
3383,4204302,Rental unit in New York · 1 bedroom · 1 bed · ...,17550546,Genevieve,Manhattan,Greenwich Village,40.732930,-73.997820,Entire home/apt,180.0,...,2014-11-09,0.02,1,0,0,No License,No rating,1,1,1
339,38529246,Rental unit in New York · ★4.78 · 2 bedrooms ·...,120810666,Rich & Patricia,Manhattan,Theater District,40.760560,-73.987060,Entire home/apt,362.0,...,2023-12-05,3.76,1,304,36,No License,4.78,2,2,1.5
16267,666022943667475224,Rental unit in Brooklyn · ★4.87 · Studio · 1 b...,402187116,James&Laureta,Brooklyn,Bedford-Stuyvesant,40.681190,-73.941300,Entire home/apt,110.0,...,2023-12-22,3.33,6,163,37,Exempt,4.87,Studio,1,1
7128,16213608,Rental unit in Queens · ★4.50 · 1 bedroom · 1 ...,83627325,Jared,Queens,Sunnyside,40.745640,-73.916480,Private room,109.0,...,2019-07-17,0.07,4,0,0,No License,4.50,1,1,1
2603,50196868,Condo in Brooklyn · 2 bedrooms · 3 beds · 2.5 ...,96056572,Xinyu,Brooklyn,Bedford-Stuyvesant,40.689410,-73.948960,Entire home/apt,180.0,...,2023-09-05,0.24,1,364,1,No License,No rating,2,3,2.5
18455,811589415141588632,Rental unit in New York · ★4.70 · Studio · 1 b...,313345834,Ruko B,Manhattan,Hell's Kitchen,40.762572,-73.997814,Entire home/apt,200.0,...,2023-11-30,1.02,1,161,10,No License,4.70,Studio,1,1
4784,942634387818750378,Rental unit in Brooklyn · ★4.62 · 2 bedrooms ·...,447283837,Shemi,Brooklyn,Prospect-Lefferts Gardens,40.660794,-73.942335,Entire home/apt,223.0,...,2023-11-26,2.77,12,179,13,No License,4.62,2,4,1


In [27]:
print("Original DataFrame shape:", df.shape)
print("DataFrame shape after filtering:", df_filtered.shape)
print(f"Removed {df.shape[0] - df_filtered.shape[0]} rows.")

Original DataFrame shape: (20758, 22)
DataFrame shape after filtering: (12548, 22)
Removed 8210 rows.
